In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up the test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'D:/DATASET/CNN/steatosis/train',  # Adjust path to your test data directory
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 38516 images belonging to 4 classes.


In [4]:
from tensorflow.keras.applications import ResNet50, DenseNet121, InceptionV3, MobileNetV2, EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def create_model(base_model, input_shape=(299, 299, 3), num_classes=5):
    base_model = base_model(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load models
models = {
    'ResNet50': create_model(ResNet50),
    'DenseNet121': create_model(DenseNet121),
    'InceptionV3': create_model(InceptionV3),
    'MobileNetV2': create_model(MobileNetV2),
    'EfficientNetB0': create_model(EfficientNetB0)
}

# You would typically load the weights for these models from your trained models, like:
# models['ResNet50'].load_weights('path_to_resnet50_weights.h5')

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Get the true labels
y_true = test_generator.classes
n_classes = 4
y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))

fpr = dict()
tpr = dict()
roc_auc = dict()

for model_name, model in models.items():
    # Predict probabilities
    predictions = model.predict(test_generator, steps=len(test_generator))
    
    # Compute ROC curve and ROC AUC for each class
    fpr[model_name] = dict()
    tpr[model_name] = dict()
    roc_auc[model_name] = dict()

    for i in range(n_classes):
        fpr[model_name][i], tpr[model_name][i], _ = roc_curve(y_true_bin[:, i], predictions[:, i])
        roc_auc[model_name][i] = auc(fpr[model_name][i], tpr[model_name][i])

1110/1204 [==========================>...] - ETA: 3:45:50

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
colors = ['aqua', 'darkorange', 'cornflowerblue', 'red', 'purple']
for i, model_name in enumerate(models.keys()):
    plt.plot(fpr[model_name][0], tpr[model_name][0], color=colors[i], lw=2, label=f'{model_name} (AUC = {roc_auc[model_name][0]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Comparison')
plt.legend(loc="lower right")
plt.show()